# Grad-CAM 히드맵 시각화

Grad-CAM output 자료를 바탕으로 여러 QR 코드 이미지의 Grad-CAM 결과를 누적하여 히트맵을 생성하는 모듈입니다.

In [4]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

In [5]:
# 환경 설정
phishing_folder = '../data/images/phishing'  # change me
model_path = '../best_model.h5'  # change me

In [6]:
# 모델 로드
model = tf.keras.models.load_model(model_path)
print("모델 로드 완료")

ModuleNotFoundError: No module named 'keras._tf_keras'

In [ ]:
# 전처리
def preprocess_image(image_path, target_size=(128, 128)):
    """이미지 전처리"""
    img = tf.keras.preprocessing.image.load_img(image_path, color_mode='grayscale', target_size=target_size)
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0
    return img_array

In [ ]:
# Guided Grad-CAM
def guided_grad_cam(model, img, layer_name):
    """Guided Grad-CAM 모델 결정 과정 시각화 함수"""
    img_tensor = tf.convert_to_tensor(img, dtype=tf.float32)
    
    grad_model = tf.keras.models.Model(
        inputs=[model.inputs],
        outputs=[model.get_layer(layer_name).output, model.output]
    )
    
    with tf.GradientTape() as tape:
        tape.watch(img_tensor)
        conv_outputs, predictions = grad_model(img_tensor)
        loss = predictions[:, 1]  # 피싱 클래스에 대한 출력
    
    grads = tape.gradient(loss, conv_outputs)[0]
    output = conv_outputs[0]
    
    weights = tf.reduce_mean(grads, axis=(0, 1))
    cam = np.zeros(output.shape[0:2], dtype=np.float32)
    
    for index, w in enumerate(weights):
        cam += w * output[:, :, index]
    
    cam = cv2.resize(cam.numpy(), (128, 128))
    cam = np.maximum(cam, 0)
    heatmap = (cam - cam.min()) / (cam.max() - cam.min())
    
    # Guided Backpropagation
    guided_model = tf.keras.models.Model(
        inputs=[model.inputs],
        outputs=[model.get_layer(layer_name).output]
    )
    
    with tf.GradientTape() as tape:
        tape.watch(img_tensor)
        outputs = guided_model(img_tensor)
    
    guided_grads = tape.gradient(outputs, img_tensor)[0]
    guided_grads = np.maximum(guided_grads, 0)  # ReLU
    
    # Guided Grad-CAM
    guided_cam = heatmap[..., np.newaxis] * guided_grads
    guided_cam = cv2.resize(guided_cam, (128, 128))
    
    return guided_cam

In [ ]:
# heat-map visualizer
def process_multiple_images(image_folder, model, layer_name='conv2d_2'):
    """여러 이미지의 Grad-CAM 결과를 누적하여 히트맵 생성"""
    accumulated_heatmap = np.zeros((128, 128))  # 모델의 입력 크기에 맞춤
    total_images = 0
    
    # 이미지 확장자 필터링
    valid_extensions = ('.png', '.jpg', '.jpeg')
    
    for filename in os.listdir(image_folder):
        if filename.lower().endswith(valid_extensions):
            try:
                image_path = os.path.join(image_folder, filename)
                img = preprocess_image(image_path)
                
                # Grad-CAM 생성
                grad_cam_result = guided_grad_cam(model, img, layer_name)
                
                # 결과 누적
                accumulated_heatmap += grad_cam_result
                total_images += 1
                
                # 진행상황 출력
                if total_images % 10 == 0:
                    print(f"처리된 이미지: {total_images}")
                
            except Exception as e:
                print(f"Error processing {filename}: {str(e)}")
    
    # 평균 히트맵 계산
    if total_images > 0:
        accumulated_heatmap /= total_images
    
    return accumulated_heatmap, total_images


def visualize_accumulated_heatmap(heatmap, total_images):
    """누적된 히트맵 시각화"""
    plt.figure(figsize=(10, 8))
    plt.imshow(heatmap, cmap='viridis')
    plt.colorbar(label='Average Activation')
    plt.title(f'Accumulated Grad-CAM Heatmap\n(Total images: {total_images})')
    plt.axis('off')
    plt.show()
    
    # 히트맵 저장
    plt.savefig('accumulated_heatmap.png', bbox_inches='tight', dpi=300)

In [ ]:
# 누적 히트맵 생성
heatmap, total = process_multiple_images(phishing_folder, model)

# 결과 시각화
visualize_accumulated_heatmap(heatmap, total)